In [3]:
#DataFrame
import pandas as pd

# Matplot
import matplotlib.pyplot as plt
%matplotlib inline

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer

# Word2vec
import gensim

# Utility
import re
import numpy as np
import os
from collections import Counter
import logging
import time
import pickle
import itertools

# Set log
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [4]:
import jieba

In [80]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from gensim.models import word2vec

In [41]:
#将单句拆分为字的list，并进行了非法字符过滤
def poem_cut(sentence):
    #@sentence：string
    #@return list of words stripped from sentence
    r1 = '[a-zA-Z0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~]+'
    r2 = '/[\u4e00-\u9fa5]+'
    #去掉虚词
    stopwords = '矣|哉|虽|既|而|何|乃|乎|其|且|若|所|为|焉|以|因|于|与|也|则|者|之|不|自|得|来|去|无|可|是|已|此|的|上|中|兮|一'
    sentence = re.sub(r1, '', sentence)
    sentence = re.sub(r2, '', sentence)
    
    result = [word.strip('，|）|：|{|}|“|”|.|！|?|。|□|·|（|\n|\r|\n\n|\xa0|\u3000').strip(stopwords) for word in sentence]
    
    return list(filter(None, result))

In [6]:
t = '1.云横秦岭家何在，雪拥蓝关马不前□。'

In [7]:
poem_cut(t)

['云', '横', '秦', '岭', '家', '在', '雪', '拥', '蓝', '关', '马', '前']

In [47]:
def readfile(path):
    #@path: stirng of data file path
    #return list of strings extracted from the txt file
    f = open(path, encoding='utf', errors='ignore')
    lines = f.readlines()
    data = []
    for line in lines:
        #去掉作者
        if re.match(r'【', line) is not None:
            continue
        if re.match(r'^\[', line) is not None:
            continue
        data.append(line) 
    f.close()
    return data    

In [48]:
def data_preprocess(data_raw):
    #@data_raw: list of strings
    #return list of single word extracted from data_raw
    #获得单词的List
    data = []
    for sentence in data_raw:
        result = poem_cut(sentence)
        if result == []:
            continue
        for word in result:
            if word == ' ':
                continue
            data.append(word)
    return data

In [96]:
#jieba分词：输入单句，过滤非法字符
def jieba_clean_text(sentence):
    #@sentence: string
    #return string after being cleaned 
    r1 = '[a-zA-Z0-9’!"#$%&\'（）()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！·[\\]^_`{|}~]+'
    r2 = '/[\u4e00-\u9fa5]+'
    #去掉虚词
    sentence = re.sub(r1, '', sentence)
    sentence = re.sub(r2, '', sentence)
    return sentence

In [50]:
def jieba_process(data):
    #@data: list of strings
    #return list of words extraced from strings and being filtered by stopwords
    stopword = '有|如|在|矣|哉|虽|既|而|何|乃|乎|其|且|若|所|为|焉|以|因|于|与|也|则|者|之|不|自|得|来|去|无|可|是|已|此|的|上|中|兮|一'
    result = []
    for sentence in data:
        clean_text = jieba_clean_text(sentence)
        string = ' '.join(jieba.cut(clean_text, cut_all = True))
        for word in string.split():
            word = word.strip('，|）|：|{|}|“|”|.|！|?|。|□|·|（|\n|\r|\n\n|\xa0|\u3000').strip(stopword)
            result.append(word)
    return list(filter(None,result))

In [51]:
def word_frequency(data):
    #@data: list of strings
    #return dict showing how many times a word appears in data
    word_dict={}
    for char in data:
        if char in word_dict:
            word_dict[char] = word_dict[char]+1
        else:
            word_dict[char] = 1
    return word_dict

In [52]:
def freq_sort(word_dict):
    #@word_dict: dict 
    #return list of sorted tuples from word_dict
    result = sorted(word_dict.items(), key=lambda d:d[1], reverse = True)
    return result

In [53]:
#虚词 停用词
# stopwords = '矣|哉|虽|既|而|何|乃|乎|其|且|若|所|为|焉|以|因|于|与|也|则|者|之|不|自|得|来|去|无|可|是|已|此|的|上|中|兮|一'

#文件路径
path_taihu = "./data/poem_taihu.txt"
path_huzhou = "./data/poem_huzhou.txt"
path_moushan = "./data/poem_moushan.txt"

In [54]:
data_taihu = readfile(path_taihu)
data_huzhou = readfile(path_huzhou)
# data_moushan = readfile(path_moushan)

In [55]:
# test1 = data_taihu[0]
# print(poem_cut(test1))
# print(data_huzhou)

In [56]:
data_hu = data_taihu + data_huzhou
print(data_hu)

['浣溪沙\n', '乐在烟波钓是闲。草堂松桂已胜攀。梢梢新月几回弯。一碧太湖三万顷，屹然相对洞庭山。况风浪起且须还。\n', '\n', '水调歌头·舣棹太湖岸\n', '舣棹太湖岸，天与水相连。垂虹亭上，五年不到故依然。洗我征尘三斗，快揖商飚千里，鸥鹭亦翩翩。身在水晶阙，真作驭风仙。\n', '望中秋，无五日，月还圆。倚栏清啸孤发，惊起壑龙眠。欲酹鸱夷西子，未办当年功业，空击五湖船。不用知余事，莼鲈正芳鲜。\n', '\n', '偈颂一百零二首\n', '西风簇浪花，太湖连底冻。冷照玉奁清，一片无瑕缝。\n', '面目分明，眼睛定动。不墯虚凝裂万差，漆桶漆桶。\n', '\n', '湖州歌九十八首 其十\n', '太湖风卷浪头高，锦柁摇摇坐不牢。\n', '靠著篷窗垂两目，船头船尾烂弓刀。\n', '山居杂诗九十首\n', '英石多斧凿，司石殊怯脆。拳然太湖出，始有岩壑意。\n', '弹窝数峰绿，欲与仇池对。永念灵壁巧，嵌空劳梦寐。\n', '\n', '点绛唇·莫倚高楼\n', '莫倚高楼，太湖西畔青山近。雁边云暝。目力随天尽。落日平芜，点点余烽烬。西风紧。乱沙成阵。双蓬鬓。\n', '\n', '江城子·春来江上打头风\n', '春来江上打头风。吼层空。卷飞蓬。多少云涛，雪浪暮江中。早是客情多感慨，烟漠漠，雨濛濛。梁溪只在太湖东。长儿童。学庞翁。谁信家书，三月不曾通。见说浙河金鼓震，何日到，羡归鸿。\n', '\n', '浪淘沙·歌阕斗清新\n', '歌阕斗清新。檀板初匀。画堂新筑太湖滨。好是黄花开应候，聊宴亲宾。上客即逢辰。况是青春。上林开宴锡尧尊。今夜素娥真解事，偏向人明。\n', '\n', '\n', '偈颂一百二十三首\n', '年来年去，去来不动乾坤。岁旧岁新，新旧一般时节。\n', '苍卞山风骨自露，太湖水彻底无波。冷冷落落，哆哆和和。\n', '门外雪消春水滑，庭前玉立桂婆娑。\n', '\n', '偈颂一百二十三首\n', '一人打毡拍板，一人吹无孔笛。梵音清雅，令人乐闻。\n', '且道是什么曲调，洞庭山脚太湖心。\n', '\n', '夏日田园杂兴\n', '中秋全景属潜夫，棹入空明看太湖。\n', '身外水天银一色，城中有此月明无。\n', '\n', '缥缈峰\n', '满载清闲一棹孤，长风相送入仙都。\n', '莫愁怀抱无消豁，缥

In [57]:
for i in range(len(data_hu)):
    if type(data_hu[i]) is not str:
        print(i)

In [58]:
type(data_hu[660])

str

In [59]:
data_Hu = data_preprocess(data_hu)

In [61]:
print(data_Hu)

['浣', '溪', '沙', '乐', '在', '烟', '波', '钓', '闲', '草', '堂', '松', '桂', '胜', '攀', '梢', '梢', '新', '月', '几', '回', '弯', '碧', '太', '湖', '三', '万', '顷', '屹', '然', '相', '对', '洞', '庭', '山', '况', '风', '浪', '起', '须', '还', '水', '调', '歌', '头', '舣', '棹', '太', '湖', '岸', '舣', '棹', '太', '湖', '岸', '天', '水', '相', '连', '垂', '虹', '亭', '五', '年', '到', '故', '依', '然', '洗', '我', '征', '尘', '三', '斗', '快', '揖', '商', '飚', '千', '里', '鸥', '鹭', '亦', '翩', '翩', '身', '在', '水', '晶', '阙', '真', '作', '驭', '风', '仙', '望', '秋', '五', '日', '月', '还', '圆', '倚', '栏', '清', '啸', '孤', '发', '惊', '起', '壑', '龙', '眠', '欲', '酹', '鸱', '夷', '西', '子', '未', '办', '当', '年', '功', '业', '空', '击', '五', '湖', '船', '用', '知', '余', '事', '莼', '鲈', '正', '芳', '鲜', '偈', '颂', '百', '零', '二', '首', '西', '风', '簇', '浪', '花', '太', '湖', '连', '底', '冻', '冷', '照', '玉', '奁', '清', '片', '瑕', '缝', '面', '目', '分', '明', '眼', '睛', '定', '动', '墯', '虚', '凝', '裂', '万', '差', '漆', '桶', '漆', '桶', '湖', '州', '歌', '九', '十', '八', '首', '十', '太', '湖', '风', '卷', '浪', '头', '高', '锦', '柁', '摇', '摇',

In [63]:
dict_freq_hu = word_frequency(data_Hu)
word_freq_hu = freq_sort(dict_freq_hu)

In [64]:
print(word_freq_hu)

[('湖', 233), ('太', 162), ('山', 124), ('人', 101), ('风', 98), ('水', 93), ('天', 76), ('有', 73), ('月', 72), ('州', 66), ('日', 61), ('云', 59), ('清', 59), ('如', 59), ('江', 58), ('溪', 57), ('生', 56), ('年', 56), ('三', 56), ('相', 51), ('在', 49), ('石', 49), ('白', 49), ('头', 47), ('西', 47), ('万', 45), ('明', 45), ('今', 44), ('春', 44), ('东', 43), ('千', 43), ('玉', 42), ('青', 42), ('夜', 41), ('首', 41), ('秋', 41), ('吴', 40), ('多', 40), ('歌', 40), ('花', 40), ('我', 38), ('君', 38), ('长', 38), ('归', 37), ('时', 37), ('烟', 36), ('见', 35), ('十', 35), ('里', 35), ('波', 35), ('平', 34), ('雪', 34), ('心', 34), ('高', 34), ('入', 34), ('舟', 34), ('空', 34), ('行', 34), ('尽', 33), ('流', 33), ('看', 33), ('龙', 33), ('谁', 32), ('过', 32), ('松', 31), ('落', 31), ('南', 31), ('望', 31), ('未', 31), ('游', 30), ('事', 30), ('客', 30), ('家', 30), ('古', 30), ('寒', 29), ('子', 29), ('下', 29), ('作', 29), ('雨', 29), ('仙', 29), ('当', 29), ('百', 28), ('飞', 28), ('到', 28), ('出', 28), ('前', 28), ('二', 28), ('五', 28), ('老', 27), ('色', 27), ('回',

In [125]:
for pair in word_freq_hu:
    if pair[0] in ['红','丹''朱','赤','绛','黑','暗','玄','乌','冥','墨','绿','碧','翠','苍','白','素','皎','皓']:
        print(pair[0] + ': ' + str(pair[1]))

白: 49
碧: 24
绿: 15
苍: 13
红: 12
墨: 9
素: 8
翠: 8
乌: 6
玄: 5
赤: 5
暗: 4
冥: 3
绛: 3
皎: 2
黑: 1


In [101]:
# l = sorted(word_freq_taihu.items(), key=lambda d:d[1], reverse = True)
# print(l)

In [102]:
# test1 = '小明硕士毕业于中国科学院计算所，后在日本京都大学深造'
# se = jieba_clean_text(data_taihu[1])
# seg_list = jieba.cut(se, cut_all = True)
# s = ' '.join(seg_list)
# word = []
# for w in s.split():
#     word.append(w)
# print(word)

In [67]:
jieba_hu = list(jieba_process(data_hu))

In [68]:
#print(jieba_taihu)

In [69]:
dict_jieba_hu = word_frequency(jieba_hu)
word_freq_jieba_hu = freq_sort(dict_jieba_hu)

In [71]:
print(word_freq_jieba_hu)

[('太湖', 153), ('湖州', 52), ('山', 51), ('人', 49), ('云', 37), ('多', 34), ('水', 34), ('溪', 33), ('我', 33), ('月', 33), ('天', 32), ('风', 31), ('君', 31), ('看', 29), ('尽', 29), ('时', 29), ('过', 29), ('谁', 29), ('清', 28), ('归', 28), ('到', 28), ('日', 26), ('生', 25), ('似', 25), ('白', 25), ('春', 25), ('望', 25), ('欲', 25), ('雪', 24), ('吴', 24), ('玉', 24), ('石', 24), ('秋', 24), ('事', 23), ('作', 23), ('见', 23), ('相', 23), ('寄', 23), ('空', 23), ('舟', 22), ('垂', 22), ('烟', 22), ('未', 22), ('波', 22), ('高', 21), ('游', 20), ('今', 20), ('树', 20), ('和', 20), ('棹', 20), ('歌', 20), ('老', 19), ('江', 19), ('夜', 19), ('亦', 19), ('苕', 19), ('碧', 19), ('入', 19), ('行', 19), ('还', 18), ('只', 18), ('年', 18), ('处', 18), ('客', 18), ('对', 17), ('梦', 17), ('前', 17), ('虹', 17), ('知', 17), ('闻', 17), ('当', 17), ('须', 17), ('寒', 17), ('松', 16), ('心', 16), ('仙', 16), ('出', 16), ('洞庭', 16), ('青', 16), ('非', 16), ('三', 16), ('长', 16), ('下', 15), ('诗', 15), ('古', 15), ('应', 15), ('后', 15), ('湖', 15), ('竹', 15), ('杂', 15), ('平',

In [72]:
#单独统计词语出现频率
two_words = []
for tu in word_freq_jieba_hu:
    if len(tu[0]) > 1:
        two_words.append(tu)
print(two_words)

[('太湖', 153), ('湖州', 52), ('洞庭', 16), ('万顷', 13), ('明月', 11), ('姑苏', 9), ('扁舟', 9), ('二首', 9), ('十二', 9), ('霅溪', 7), ('松江', 7), ('湖东', 7), ('湖岸', 7), ('吴江', 7), ('四十', 7), ('平生', 7), ('三首', 7), ('湖西', 7), ('西子', 7), ('水调歌头', 7), ('湖水', 7), ('三万', 7), ('依然', 6), ('今日', 6), ('天下', 6), ('西风', 6), ('风吹', 6), ('百年', 6), ('百二', 6), ('分明', 6), ('千里', 6), ('当年', 6), ('古今', 5), ('回首', 5), ('千顷', 5), ('人生', 5), ('青山', 5), ('二十', 5), ('君不见', 5), ('长桥', 5), ('东南', 5), ('十年', 5), ('吴兴', 5), ('江南', 5), ('山色', 5), ('风雨', 5), ('落日', 5), ('蛟龙', 4), ('玉山', 4), ('岩寺', 4), ('草堂', 4), ('船头', 4), ('灵岩寺', 4), ('腐儒', 4), ('百花', 4), ('山水', 4), ('风月', 4), ('万里', 4), ('画舫', 4), ('蓬莱', 4), ('相思', 4), ('山川', 4), ('江东', 4), ('山前', 4), ('倒影', 4), ('三月', 4), ('仙人', 4), ('尘土', 4), ('水晶', 4), ('十三', 4), ('江湖', 4), ('五湖', 4), ('惊起', 4), ('四首', 4), ('汀洲', 4), ('风流', 4), ('天目', 4), ('日月', 4), ('相逢', 4), ('几回', 4), ('青天', 4), ('渺然', 4), ('东风', 4), ('太湖石', 4), ('潇洒', 3), ('眼见', 3), ('惠山', 3), ('土风', 3), ('尽头', 3), ('贿赂', 3)

In [142]:
# with open('./data/poem_taihu.txt',encoding='utf-8') as f1:
#     with open('./data/poem_huzhou.txt',encoding='utf-8') as f2:
#         document1 = f1.read()
#         document2 = f2.read()
#         document = document1 + document2
#         document = jieba_clean_text(document)
#         document_cut = jieba.cut(document)
#         result = ' '.join(document_cut)
#         with open('./hu_segment.txt', 'w',encoding="utf-8") as f3:
#             f3.write(result)

In [179]:
single_word = ''
for sentence in data_hu:
    stopword = '[有如在矣哉虽既而何乃乎其且若所为焉以因于与也则者之不自得来去无可是已此的上中兮一二三四五六七八九十]'
    sentence = re.sub(stopword, '', sentence)
    clean_text = jieba_clean_text(sentence)
    single_word += ' '.join(clean_text)
with open('./hu_segment.txt', 'w', encoding="utf-8") as f:
    f.write(single_word)

In [180]:
print(single_word)

浣 溪 沙 
乐 烟 波 钓 闲 草 堂 松 桂 胜 攀 梢 梢 新 月 几 回 弯 碧 太 湖 万 顷 屹 然 相 对 洞 庭 山 况 风 浪 起 须 还 

水 调 歌 头 舣 棹 太 湖 岸 
舣 棹 太 湖 岸 天 水 相 连 垂 虹 亭 年 到 故 依 然 洗 我 征 尘 斗 快 揖 商 飚 千 里 鸥 鹭 亦 翩 翩 身 水 晶 阙 真 作 驭 风 仙 
望 秋 日 月 还 圆 倚 栏 清 啸 孤 发 惊 起 壑 龙 眠 欲 酹 鸱 夷 西 子 未 办 当 年 功 业 空 击 湖 船 用 知 余 事 莼 鲈 正 芳 鲜 

偈 颂 百 零 首 
西 风 簇 浪 花 太 湖 连 底 冻 冷 照 玉 奁 清 片 瑕 缝 
面 目 分 明 眼 睛 定 动 墯 虚 凝 裂 万 差 漆 桶 漆 桶 

湖 州 歌 首   
太 湖 风 卷 浪 头 高 锦 柁 摇 摇 坐 牢 
靠 著 篷 窗 垂 两 目 船 头 船 尾 烂 弓 刀 
山 居 杂 诗 首 
英 石 多 斧 凿 司 石 殊 怯 脆 拳 然 太 湖 出 始 岩 壑 意 
弹 窝 数 峰 绿 欲 仇 池 对 永 念 灵 壁 巧 嵌 空 劳 梦 寐 

点 绛 唇 莫 倚 高 楼 
莫 倚 高 楼 太 湖 西 畔 青 山 近 雁 边 云 暝 目 力 随 天 尽 落 日 平 芜 点 点 余 烽 烬 西 风 紧 乱 沙 成 阵 双 蓬 鬓 

江 城 子 春 江 打 头 风 
春 江 打 头 风 吼 层 空 卷 飞 蓬 多 少 云 涛 雪 浪 暮 江 早 客 情 多 感 慨 烟 漠 漠 雨 濛 濛 梁 溪 只 太 湖 东 长 儿 童 学 庞 翁 谁 信 家 书 月 曾 通 见 说 浙 河 金 鼓 震 日 到 羡 归 鸿 

浪 淘 沙 歌 阕 斗 清 新 
歌 阕 斗 清 新 檀 板 初 匀 画 堂 新 筑 太 湖 滨 好 黄 花 开 应 候 聊 宴 亲 宾 客 即 逢 辰 况 青 春 林 开 宴 锡 尧 尊 今 夜 素 娥 真 解 事 偏 向 人 明 


偈 颂 百 首 
年 年 动 乾 坤 岁 旧 岁 新 新 旧 般 时 节 
苍 卞 山 风 骨 露 太 湖 水 彻 底 波 冷 冷 落 落 哆 哆 和 和 
门 外 雪 消 春 水 滑 庭 前 玉 立 桂 婆 娑 

偈 颂 百 首 
人 打 毡 拍 板 

In [181]:
#加载语料
sentences = word2vec.LineSentence('./hu_segment.txt')

In [182]:

#训练语料
path = get_tmpfile("word2vec_hu.model") #创建临时文件
model = word2vec.Word2Vec(sentences, hs=1,min_count=2,window=5,size=100)
# model.save("word2vec.model")
# model = Word2Vec.load("word2vec.model")


2020-08-21 13:43:44,654 : INFO : collecting all words and their counts
2020-08-21 13:43:44,664 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-08-21 13:43:44,672 : INFO : collected 2393 word types from a corpus of 12902 raw words and 832 sentences
2020-08-21 13:43:44,673 : INFO : Loading a fresh vocabulary
2020-08-21 13:43:44,678 : INFO : effective_min_count=2 retains 1531 unique words (63% of original 2393, drops 862)
2020-08-21 13:43:44,679 : INFO : effective_min_count=2 leaves 12040 word corpus (93% of original 12902, drops 862)
2020-08-21 13:43:44,686 : INFO : deleting the raw counts dictionary of 2393 items
2020-08-21 13:43:44,687 : INFO : sample=0.001 downsamples 59 most-common words
2020-08-21 13:43:44,688 : INFO : downsampling leaves estimated 11087 word corpus (92.1% of prior 12040)
2020-08-21 13:43:44,691 : INFO : constructing a huffman tree from 1531 words
2020-08-21 13:43:44,733 : INFO : built huffman tree with maximum node depth 13
2020-08-2

In [185]:
for key in model.wv.similar_by_word('月', topn =20):
    print(key)

('水', 0.989631712436676)
('回', 0.9882352352142334)
('新', 0.9861561059951782)
('山', 0.9859026670455933)
('太', 0.9857019782066345)
('南', 0.9856430888175964)
('明', 0.9852582216262817)
('吴', 0.985234260559082)
('头', 0.9851284027099609)
('尽', 0.9848151206970215)
('船', 0.9847450852394104)
('谁', 0.9846909046173096)
('风', 0.9846093654632568)
('千', 0.9845812320709229)
('日', 0.98457932472229)
('溪', 0.9842718839645386)
('松', 0.9841411709785461)
('余', 0.983986496925354)
('雪', 0.9838926792144775)
('前', 0.9837945699691772)
